In [ ]:
# Install OpenNMT-py 3.x
!pip3 install OpenNMT-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 29.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Open the folder where you saved your prepapred datasets from the first exercise
# You might need to mount your Google Drive first
%cd /content/drive/MyDrive/NLP2/fine_tuned/it_to_en/nmt/
!ls

/content/drive/.shortcut-targets-by-id/15ST_GbtQNVxa6lxoGBZxd99QBIwzy53D/NLP2/fine_tuned/it_to_en/nmt
Europarl.en-it.en
Europarl.en-it.en-filtered.en
Europarl.en-it.en-filtered.en.subword
Europarl.en-it.en-filtered.en.subword.dev
Europarl.en-it.en-filtered.en.subword.test
Europarl.en-it.en-filtered.en.subword.train
Europarl.en-it.it
Europarl.en-it.it-filtered.it
Europarl.en-it.it-filtered.it.subword
Europarl.en-it.it-filtered.it.subword.dev
Europarl.en-it.it-filtered.it.subword.test
Europarl.en-it.it-filtered.it.subword.train
Europarl.en-it.xml
LICENSE
MT-Preparation
README
source.model
source.vocab
target.model
target.vocab


In [ ]:
#BIDIRECTIONAL RECURENT
config_brnn = '''# config_brnn.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: Europarl.en-it.it-filtered.it.subword.train
        path_tgt: Europarl.en-it.en-filtered.en.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: Europarl.en-it.it-filtered.it.subword.dev
        path_tgt: Europarl.en-it.en-filtered.en.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.fren

# Stop training if it does not imporve after n validations
early_stopping: 3

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 5000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 5000

# Default: 10000 - Run validation after n steps
valid_steps: 500

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 2000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]


# Model
encoder_type: brnn
word_vec_size: 128
hidden_size: 512
layers: 1

# Optimization
optim: adagrad
learning_rate: 0.15
adagrad_accumulator_init: 0.1
max_grad_norm: 2

batch_size: 16
dropout: 0.0

'''

with open("config_brnn.yaml", "w+") as config_brnn_yaml:
  config_brnn_yaml.write(config_brnn)

In [ ]:
# Find the number of CPUs/cores on the machine
!nproc --all

12


In [ ]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config_brnn.yaml -n_sample -1 -num_threads 12

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-24 13:48:05,612 INFO] Counter vocab from -1 samples.
[2023-06-24 13:48:05,612 INFO] n_sample=-1: Build vocab on full datasets.
[2023-06-24 13:48:23,569 INFO] * Transform statistics for corpus_1(8.33%):
			* FilterTooLongStats(filtered=87)

[2023-06-24 13:48:23,569 INFO] * Transform statistics for corpus_1(8.33%):
			* FilterTooLongStats(filtered=71)

[2023-06-24 13:48:23,570 INFO] * Transform statistics for corpus_1(8.33%):
			* FilterTooLongStats(filtered=85)

[2023-06-24 13:48:23,571 INFO] * Transform statistics for corpus_1(8.33%):
			* FilterTooLongStats(filtered=90)

[2023-06-24 13:48:23,572 INFO] * Transform statistics for corpus_1(8.33%):
			* FilterTooLongStats(filtered=74)

[2023-06-24 13:48:23,577 INFO] * Transform statistics for corpus_1(8.33%):
			* FilterTooLongStats(filtered=91)

[2023-06-24 13:48:23,579 INFO] * Transform statistics for corpus_1(8.33%):
			* FilterTooLongStats(filtered=79)

[20

In [ ]:
# Check if the GPU is active
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-d3c65211-f9d0-2bfa-58f7-99bf6332d166)


In [ ]:
# Check if the GPU is visable to PyTorch

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
NVIDIA A100-SXM4-40GB
Free GPU memory: 40096.5625 out of: 40513.5625


In [ ]:
# Train the NMT model
!onmt_train -config config_brnn.yaml

[2023-06-24 13:51:11,291 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-24 13:51:11,292 INFO] Parsed 2 corpora from -data.
[2023-06-24 13:51:11,292 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-24 13:51:11,606 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', ',', '▁di', '.', "'", '▁e', '▁che']
[2023-06-24 13:51:11,607 INFO] The decoder start token is: <s>
[2023-06-24 13:51:11,607 INFO] Building model...
[2023-06-24 13:51:12,186 INFO] Switching model to float32 for amp/apex_amp
[2023-06-24 13:51:12,187 INFO] Non quantized layer compute is fp32
[2023-06-24 13:51:12,350 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(49440, 128, padding_idx=1)
        )
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (rnn): LSTM(128, 256, batch_first=True, bidirectional=True

In [ ]:
# Translate the "subworded" source file of the test dataset
# Change the model name, if needed.
!onmt_translate -model models/model.fren_step_5000.pt -src Europarl.en-it.it-filtered.it.subword.test -output Europarl.en.translated -gpu 0 -min_length 1

[2023-06-24 14:11:35,974 INFO] Loading checkpoint from models/model.fren_step_5000.pt
[2023-06-24 14:11:36,660 INFO] Loading data into the model
[2023-06-24 14:12:29,487 INFO] PRED SCORE: -1.4868, PRED PPL: 4.42 NB SENTENCES: 4000


In [ ]:
# If needed install/update sentencepiece
!pip3 install --upgrade -q sentencepiece

# Desubword the translation file
!python3 MT-Preparation/subwording/3-desubword.py target.model Europarl.en.translated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.2 MB/s eta 0:00:00
Done desubwording! Output: Europarl.en.translated.desubword


In [ ]:
# Desubword the target file (reference) of the test dataset
# Note: You might as well have split files *before* subwording during dataset preperation,
# but sometimes datasets have tokeniztion issues, so this way you are sure the file is really untokenized.
!python3 MT-Preparation/subwording/3-desubword.py target.model Europarl.en-it.en-filtered.en.subword.test

Done desubwording! Output: Europarl.en-it.en-filtered.en.subword.test.desubword


In [ ]:
# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

--2023-06-24 14:13:50--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2023-06-24 14:13:50 (43.2 MB/s) - ‘compute-bleu.py’ saved [957/957]



In [ ]:
# Install sacrebleu
!pip3 install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Evaluate the translation (without subwording)
!python3 compute-bleu.py Europarl.en-it.en-filtered.en.subword.test.desubword Europarl.en.translated.desubword

Reference 1st sentence: We must ensure that there are no monopolistic conditions and that there are clear rules for calls for tender in order to involve users.
MTed 1st sentence: It must be able to be able to deal that the European Union should be able to be able to deal with the European Union.
BLEU:  2.643207991873375
